In [18]:
'''
Based on :

FINAL_LookupTables_US_ITC_HTS_Codes_2015-2017.xlsx
https://docs.google.com/spreadsheets/d/1jAx6VSln3-sCxI7R1TDtAVSZjzVsH36MpAq3yKa4e7o/edit#gid=1901895138

'''
import pandas as pd
import re
import textacy
import spacy
nlp = spacy.load('en')
import math
import gen_kw_2
import gen_kw_2
import filter_hs_codes
import collections
import re
import numpy as np

In [2]:

'''
Files
1501_hts_delimited.txt
hts_2016_basic_delimited.csv
hts_2017_preliminary_csv.csv
hts_2018_basic_csv.csv
'''

EXPERIMENT = True

def get_file_list_1():
    files = {
        'hts_2018_basic_csv.csv': 'hs_codes_file_4_2.csv'
    }
    return files

def get_file_list_2():
    files = [ 'hs_codes_file_4_2.csv' ]
    return files

# ---------------------------- #
# def get_file_list_2():
#     global EXPERIMENT
#     files = [
#         'hs_codes_file_1.csv',
#         'hs_codes_file_2.csv',
#         'hs_codes_file_3.csv',
#         'hs_codes_file_4.csv'
#     ]
#     if EXPERIMENT == True:
#         files = [
#             'hs_codes_file_1_1.csv',
#             'hs_codes_file_2_1.csv',
#             'hs_codes_file_3_1.csv'
# #             'hs_codes_file_4_1.csv'
#         ]   
#     return files
# ---------------------------- #

In [3]:
def process_chg_file(input_file, op_file):
    parts = input_file.split('.')
    file_ext = parts[1]

    if file_ext == 'txt':
        df = pd.read_csv(input_file, sep='|', encoding="ISO-8859-1")
    else:
        df = pd.read_csv(input_file, encoding="ISO-8859-1")

    def clean_description(row):
        if row is None or row['Description'] is None:
            return None
        text = str(row['Description'])
        if len(text) == 0:
            return None
#         pattern = re.compile(r'<[^>]+>')
#         text = pattern.sub('', text)
        strip_chars = ['\t', ' ', '\t\n', ':', ',', ';']
        for s in strip_chars:
            text = text.strip(s)
        text = text.replace(',', ' && ')
        text = text.replace(' and ', ' && ')
        return text

    op_path = op_file
    df['Description'] = df.apply(clean_description, axis=1)
    delete_cols = [
        'Unit of Quantity',
        'General Rate of Duty',
        'Special Rate of Duty',
        'Column 2 Rate of Duty',
        'Col. 1 Rate',
        'Special Rate',
        'Col. 2 Rate',
        'Footnote / Comment',
        'Unnamed: 2'
    ]

    for d in delete_cols:
        try:
            del df[d]
        except:
            pass
    # in case of 'HTS No.', 'Stat Suffix'  - combine them into 'hs_code'
    # also rename Level as Indent

    try:
        df = df.rename(columns={'Level': 'Indent'})
    except:
        pass

    def combine_hscode(row):
        h1 = row['HTS No.']
        h2 = row['Stat Suffix']
        if math.isnan(h2) == False and (int(h2)) > 0:
            res = h1 + '.' + str(int(h2))
        else:
            res = h1
        return res

    df_col_names = list(df.columns)

    if ('HTS No.' in df_col_names) and ('Stat Suffix' in df_col_names):
        df['hs_code'] = df.apply(combine_hscode, axis=1)
        del df['HTS No.']
        del df['Stat Suffix']
    else:
        try:
            df = df.rename(columns={'HTS Number': 'hs_code'})
        except:
            pass

    print(df.columns)
    df = filter_hs_codes.filter_df(df,level =1 )
    df.to_csv(op_path,index=False)
    print('-----------------')
    return

# ----------------------------------- #

# Clean and parse from txt/csv to csv. Place files in csv with new names

def main_1():
    files = get_file_list_1()
    for inp, op in files.items():
        process_chg_file(inp, op)
# ----------------------------------- #

main_1()

Index(['hs_code', 'Indent', 'Description'], dtype='object')
-----------------


Extract scientific names
------

In [4]:
def get_sc_names(file):
    df = pd.read_csv(file)
    df['sc_names'] = None
    for i,row in df.iterrows():
        desc_text = row['Description']
        if desc_text is None or type(desc_text) != str:
            continue
        res_list = [] 
        patterns = [
            '\<\S+\>[A-Z][a-z]+\<\S+\> \<\S+\>[a-z]+\<\S+\>',
            '\<\S+\>[A-Z][a-z]+ [a-z]+\<\S+\>',
            '\<\S+\>[A-Z][a-z]+\s*\<\S+\> [a-z]+\.',
            '\<\S+\>[A-Z][a-z]+\<\S+\> [a-z]+\.',
            '\<\S+\> [A-Z][a-z]+\<\S+\> [a-z]+\.',
            '\<\S+\> [A-Z][a-z]+\ <\S+\>  [a-z]+\.',
            '\<\S+\>[A-Z][a-z]+\ <\S+\>spp',
        ]
        
        for pattern in patterns :
            search_res = re.findall(pattern,desc_text)
            if len(search_res)> 0 :
#                 print (' -->' , desc_text)
#                 print (search_res)
                for grp in range(len(search_res)) :
                    _res = search_res[grp]
                    _res = ''.join(_res)
                    _pattern = re.compile(r'<[^>]+>')
                    text = _pattern.sub('', _res)
                    text = text.strip()
                    res_list.append(text)
               
        if len(res_list)>0 :
            res = ';'.join(res_list)
            print(res)
            df.loc[i,'sc_names'] = res
        pattern = re.compile(r'<[^>]+>')
        text = pattern.sub('', desc_text)   
        df.loc[i,'Description'] = text
        
    df.to_csv(file,index=False)
    return
            
def main_2():
    files = get_file_list_2()
    print(files)
    for inp in files :
        get_sc_names(inp)
        
# ----------------------------------- #  


main_2()

['hs_codes_file_4_2.csv']
Pinus spp.;Pinus spp.
Pinus taeda;Pinus palustris;Pinus rigida;Pinus echinata;Pinus elliottii;Pinus virginiana
Pinus ponderosa
Picea spp.;Picea spp.
Pseudotsuga menziesii
Tsuga heterophylla
Thuja plicata
Quercus spp.;Quercus spp.
Fagus spp.;Fagus spp.
Betula spp.;Betula spp.
Fraxinus spp.;Fraxinus spp.
Alnus rubra
Prunus spp.;Prunus spp.
Acer spp.;Acer spp.
Liriodendron tulipifera
Juglans spp.;Juglans spp.
Paulownia spp.;Paulownia spp.
Picea sitchensis
Pinus resinosa
Pinus contorta
Pinus taeda;Pinus palustris;Pinus rigida;Pinus echinata;Pinus elliottii;Pinus virginiana
Pinus ponderosa
Pseudotsuga menziesii
Abies spp.;Abies spp.
Tsuga spp.;Tsuga spp.
Larix spp.;Larix spp.
Chamaecyparis nootkatensis
Thuja spp.;Juniperus spp.;Chamaecyparis spp.;Cupressus spp.;Libocedrus spp.;Thuja spp.;Juniperus spp.;Chamaecyparis spp.;Cupressus spp.;Libocedrus spp.
Sequoia sempervirens
Swietenia spp.;Swietenia spp.
Ochroma lagopus
Khaya spp.;Khaya spp.
Aningeria spp.;Aningeria s

In [7]:
def clean_sc_names(file):
    df = pd.read_csv(file,index_col=None)
    
    for i,cur_row in df.iterrows():
        _res =  cur_row['sc_names']
        res = []
        if type(_res)== str :
            _res = _res.split(';')
            # if only one word - add in spp
            # also if spp -> replace with spp.
            for _r in _res:
                _r = _r.strip()
                if ' ' not in _r:
                    tmp = _r + ' spp.'
                    res.append(tmp)
                elif 'spp' in _r and 'spp.' not in _r:
                    tmp = _r.replace('spp','spp.')
                    res.append(tmp)
                else:
                    res.append(_r)
            res = list(set(res))
            res = ';'.join(res)
            df.at[i, 'sc_names'] = res
    df.to_csv(file,index=False)
        

def main_5():
    files = get_file_list_2()
    for inp in files :
        clean_sc_names(inp)

main_5()  

Write out the complete descriptions
------

In [8]:
def main_3_aux(file):

    def gen_aux_desc(df, idx):
        min_idx = 0
        neg_keyword = 'Other'
        res = None
        cur_row = df.loc[idx]
        indent_attr = 'Indent'
        try:
            cur_indent = cur_row['Indent']
        except:
            cur_indent = cur_row['Level']
            indent_attr = 'Level'

        cur_desc = cur_row['Description']
        is_neg_keyword = False

        if type(cur_desc) != str:
            cur_desc = ''

        if cur_desc == neg_keyword:
            is_neg_keyword = True

        def format_desc(s):
            return '[ ' + s + ' ]'

        if cur_indent == 0:
            res = format_desc(cur_desc)
        else:
            # go up and fetch
            cur_idx = idx - 1
            if is_neg_keyword:
                res_parts = []
                while cur_idx >= min_idx:
                    r = df.loc[cur_idx]
                    # Find entry in same level
                    if r[indent_attr] == cur_indent:
                        # add in what the thing is not
                        d3 = r['Description']
                        if type(d3) == str and len(d3) > 0:
                            t = '!' + format_desc(d3)
                            res_parts.append(t)

                    elif r[indent_attr] == cur_indent - 1:
                        fd = r['full_desc']
                        res = ' && '.join(res_parts)
                        res = format_desc(res)
                        res = fd + ' && ' + res
                        res = format_desc(res)
                        break

                    cur_idx -= 1

            else:
                while cur_idx >= min_idx:
                    r = df.loc[cur_idx]
                    dsc = r['full_desc']
                    if r[indent_attr] == cur_indent - 1:
                        res = dsc + ' && ' + format_desc(cur_desc)
                        break
                    else:
                        cur_idx -= 1
        return res

    def generate_desc(df):
        df.at[:, 'full_desc'] = None
        new_df = df.copy(deep=True)
        for i, row in df.iterrows():
            _desc = gen_aux_desc(new_df, i)
            new_df.at[i, 'full_desc'] = _desc
        return new_df


    df = pd.read_csv(file,index_col=None)
    df = generate_desc(df)
    try:
        del df['Unnamed: 0']
    except:
        pass
    print('main_3_aux', file)
    print('-----------------')
    df.to_csv(file,index=False)

    
def main_3():
    files = get_file_list_2()
    for inp in files :
        main_3_aux(inp)
        
main_3()


main_3_aux hs_codes_file_4_2.csv
-----------------


Add in keywords - unigrams, bi-grams and trigrams
----

In [9]:
def add_in_kws(file):

    df = pd.read_csv(file,index_col=None)
    df = gen_kw_2.add_in_kws(df)
    df.to_csv(file,index=False)
    print('add_in_kws', file)
    print('-----------------')
    return


def main_4():
    files = get_file_list_2()
    for inp in files :
        add_in_kws(inp)

main_4()

add_in_kws hs_codes_file_4_2.csv
-----------------


In [30]:
def get_hs_code_6(row):

    if row['hs_code'] is None or type(row['hs_code']) == float :
        return None
    else:
        h = str(row['hs_code'])
        h = h.replace('.','',10)
        if len(h) > 6:
            h = h[:6]
        h = str(int(h))
        return h

def main_5():
    files = get_file_list_2()
    for f in files :
        df = pd.read_csv(f,index_col=None)
        df['hscode_6'] = None
        df['hscode_6'] = df.apply(get_hs_code_6, axis=1)
        df.to_csv(f,index=False)
        
main_5()


In [41]:
# let us do an approximation

def main_6():
    files = get_file_list_2()
    f = files[0]
    df = pd.read_csv(f,index_col=None)
    set_hscode_6 =  set(df['hscode_6'].dropna())
    result_df = pd.DataFrame(columns= ['hscode_6','keywords','sc_names'])
    
    set_hscode_6 = sorted(set_hscode_6)
    for _hs6 in set_hscode_6:
        hscode = _hs6
        _hs6 = str(int(_hs6))
        if len(_hs6)<6 :
            continue
        _df = df.loc[df['hscode_6'] == hscode]
        
        kw_store = []
        scn = []
        for i,row in _df.iterrows():
            _k = row['pos_kw'].split(';')
            kw_store.extend(_k)
            _scn =  row['sc_names']
            if type(_scn) == str:
                _scn = _scn.split(';')
                scn.extend(_scn)
            
        kw_store = set(kw_store)
        kw_store = [ str(t).lower() for t in kw_store ]
        sc_names = list(set(scn))
        
        kw = ';'.join(kw_store)
        scn = ';'.join(sc_names)
        print(_hs6, kw, scn)
        _dict = {
            'hscode_6' : _hs6,
            'keywords' : kw,
            'sc_names' : scn
        }
        result_df = result_df.append(_dict, ignore_index=True)
    result_df.to_csv('HSCode_6_data_v0.csv',index=False)
    return

main_6()

440110 faggot;billet;briquette;chip;particle;twig;scrap;similar form;fuel wood;wood waste;log 
440121 faggot;wood;billet;briquette;chip;particle;twig;scrap;similar form;coniferous;fuel wood;wood waste;log 
440122 faggot;wood;billet;briquette;chip;particle;nonconiferous;twig;scrap;similar form;fuel wood;wood waste;log 
440131 faggot;sawdust;billet;briquette;chip;particle;twig;scrap;similar form;wood pellet;fuel wood;wood waste;log 
440139 faggot;sawdust;material;billet;briquette;chip;particle;fire log;wax;twig;scrap;similar form;log compose;artificial fire;fuel wood;wood waste;log 
440210 nut charcoal;bamboo;wood charcoal;shell 
440290 nut charcoal;wood charcoal;shell 
440310 creosote;post;paint;telegraph;poles;preservative;squared;telephone;strip;electrical power;power pole;rough;pile;fence post;sap wood;bark 
440320 pinus spp;cedar;pitch pine;pinus taeda;pinus echinata;pseudotsuga menziesii;thuja plicata;western hemlock;logs;tsuga heterophylla;coniferous;long leaf;short leaf;yellow pi